<a href="https://colab.research.google.com/github/lephuocdat2000/truyvandaphuongtien/blob/master/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#đọc file data
import pandas as pd
import numpy as np
import math
import re
book_info=pd.read_json('/content/Book_info.json',encoding='utf-8')
book_info.head()

,NO,BIBID,AUTHOR,TITLE,PUBLISHER,PUBDATE,COUNTHOLDING,TOPIC,CONTENT
0,0,4386,Trần Hồng Nghi,Hướng dẫn thực hành phát triển ứng dụn...,Đại học Quốc gia Tp. Hồ Chí Min,2019,100,"[Mạng máy tính, Hệ điều hành Android]",Cung cấp những kiến thức cơ bản về công ...
1,1,4385,Lâm Đức Khải,Thiết kế luận lý số 2,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Kỹ thuật máy tính],None
2,2,4384,Cao Thị Nhạn,Phân tích thiết kế Hệ thống thông tin,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,[Hệ thống thông tin],None
3,3,4382,Đỗ Phúc,Phân tích Dữ liệu lớn,Đại học Quốc gia Thành phố Hồ Chí Min,2019,100,"[Hệ thống thông tin, Giáo trình]",None
4,4,4365,Nguyễn Anh Tuấn,Hệ điều hành Linux,Đại học Quốc gia TPHC,2019,100,[Hệ điều hành Linux],None


In [ ]:
#Xây dựng tập từ điển 
def Dic_building(a):
  contents=book_info[a].to_numpy()
  lst_contents=[]
  dictionary=set()
  if (a!='TOPIC'):
    for content in contents:
       if content==None: continue
       words = content.replace('"', '').replace('.', '').replace("'","").split()
       lst_contents.append(words)
       dictionary.update(words)
  else:
    for content in contents_TOPIC:
       for i in content:
         words=i.replace('"', '').replace('.', '').replace("'","").split()
         lst_contents.append(words)
         dictionary.update(words)
  dictionary = list(dictionary)
  return contents,lst_contents,dictionary
#Xây dựng Inverted File
def build_inverted_files(dictionary,lst_contents):
   inv_files=dict()
   for k,word in enumerate(dictionary):
      inv_files[word] = set()
      for i,content in enumerate(lst_contents):
         if word in content:
            count=content.count(word)
            tf=count/len(content)
            tup_word_tf=(i,tf)
            inv_files[word].add(tup_word_tf)
   return inv_files

#3. Tính toán tf-idf
def calcu_word_tfidf(inv_files,lst_contents):
    tf_idf_arr=np.zeros(len(inv_files))
    total_content=len(lst_contents)
    for i,word in enumerate(inv_files):
        k=len(inv_files[word])
        idf=math.log(total_content/k)
        for tup in inv_files[word]:
            temp=list(tup)
            temp[1]*=idf
            tup=tuple(temp)
def Build_invfile(a):
   contents,lst_contents,dictionary = Dic_building(a)
   inv_files = build_inverted_files(dictionary,lst_contents)
   calcu_word_tfidf(inv_files,lst_contents)
   return contents,lst_contents,dictionary,inv_files

In [ ]:
contents_TOPIC,lst_contents_TOPIC,dic_TOPIC,inv_files_TOPIC, = Build_invfile('TOPIC')
contents_TITLE,lst_contents_TITLE,dic_TITLE,inv_files_TITLE = Build_invfile('TITLE')
contents_AUTHOR,lst_contents_AUTHOR,dic_AUTHOR,inv_files_AUTHOR = Build_invfile('AUTHOR')
contents_CONTENT,lst_contents_CONTENT,dic_CONTENT,inv_files_CONTENT = Build_invfile('CONTENT')

In [ ]:
query=str(input())
def Create_tfidf_query(query,lst_content,inv_files):
   query_words=re.findall("(\w+)", query)
   dict_query=set()
   dict_query.update(query_words)
   dict_query=list(dict_query)
   tfidf_query=np.zeros(len(dict_query))
   total_content=len(lst_contents)
   for i,word in enumerate(dict_query):
     try:
        idf=1+np.log(total_content/(len(inv_files[word])))
     except: 
        idf=1+np.log(total_content)
        tfidf_query[i]=query_words.count(word)*idf/len(query_words)
#print("tfidf_query:",tfidf_query)
   query_words=re.findall("(\w+)", query)


Lê Phước Đạt
